### Import Libraries

In [243]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [244]:
print(tf.__version__)

2.1.0


# Dataset

In [245]:
DATA_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data"

csv_file = keras.utils.get_file(fname="wdbc.data", origin=DATA_URL)

In [246]:
# Make numpy values easier to read.
np.set_printoptions(precision=6, suppress=True)

In [247]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = pd.read_csv(csv_file)

df.columns = ['id', 'diagnosis', 'radius', 'texture', 'perimeter', 'area', 'smoothness','compactness', 'concavity',
              'concave_points', 'symmetry', 'fractal_dimension', 'radius_se','texture_se', 'perimeter_se', 'area_se',
              'smoothness_se', 'compactness_se', 'concavity_se', 'concave_points_se', 'symmetry_se', 'fractal_dimension_se',
              'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst',
              'concavity_worst', 'concave_points_worst', 'symmetry_worst', 'fractal_dimension_worst']
df.diagnosis.replace(to_replace=['M', 'B'], value=[1, 0], inplace=True)

df.head()

,id,diagnosis,radius,texture,perimeter,area,smoothness,compactness,concavity,concave_points,symmetry,fractal_dimension,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave_points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
1,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
2,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
3,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678
4,843786,1,12.45,15.70,82.57,477.1,0.12780,0.17000,0.1578,0.08089,0.2087,0.07613,0.3345,0.8902,2.217,27.19,0.007510,0.03345,0.03672,0.01137,0.02165,0.005082,15.47,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440


### Data normalization

In [248]:
diagnosis = df.pop('diagnosis')
id_dataset = df.pop('id')

min_max_scaler = MinMaxScaler()
min_max_scaler_m1 = MinMaxScaler(feature_range=(-1,1))

# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(df)
x_scaled_m1 = min_max_scaler_m1.fit_transform(df)

# Run the normalizer on the dataframe
df_normalized = pd.DataFrame(x_scaled)
df_normalized_m1 = pd.DataFrame(x_scaled_m1)

In [249]:
# View the dataframe
df_normalized

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,0.643144,0.272574,0.615783,0.501591,0.289880,0.181768,0.203608,0.348757,0.379798,0.141323,0.156437,0.082589,0.124440,0.125660,0.119387,0.081323,0.046970,0.253836,0.084539,0.091110,0.606901,0.303571,0.539818,0.435214,0.347553,0.154563,0.192971,0.639175,0.233590,0.222878
1,0.601496,0.390260,0.595743,0.449417,0.514309,0.431017,0.462512,0.635686,0.509596,0.211247,0.229622,0.094303,0.180370,0.162922,0.150831,0.283955,0.096768,0.389847,0.205690,0.127006,0.556386,0.360075,0.508442,0.374508,0.483590,0.385375,0.359744,0.835052,0.403706,0.213433
2,0.210090,0.360839,0.233501,0.102906,0.811321,0.811361,0.565604,0.522863,0.776263,1.000000,0.139091,0.175875,0.126655,0.038155,0.251453,0.543215,0.142955,0.353665,0.728148,0.287205,0.248310,0.385928,0.241347,0.094008,0.915472,0.814012,0.548642,0.884880,1.000000,0.773711
3,0.629893,0.156578,0.630986,0.489290,0.430351,0.347893,0.463918,0.518390,0.378283,0.186816,0.233822,0.093065,0.220563,0.163688,0.332359,0.167918,0.143636,0.357075,0.136179,0.145800,0.519744,0.123934,0.506948,0.341575,0.437364,0.172415,0.319489,0.558419,0.157500,0.142595
4,0.258839,0.202570,0.267984,0.141506,0.678613,0.461996,0.369728,0.402038,0.518687,0.551179,0.080753,0.117132,0.068793,0.038080,0.197063,0.234311,0.092727,0.215382,0.193730,0.144660,0.268232,0.312633,0.263908,0.136748,0.712739,0.482784,0.427716,0.598282,0.477035,0.454939
5,0.533343,0.347311,0.523875,0.380276,0.379164,0.274891,0.264058,0.367793,0.370707,0.157119,0.121383,0.091275,0.114169,0.087987,0.088418,0.086881,0.056919,0.196818,0.081725,0.044367,0.531839,0.416844,0.511928,0.349194,0.482269,0.223448,0.302236,0.663918,0.295289,0.187853
6,0.318472,0.376057,0.320710,0.184263,0.598267,0.445126,0.219447,0.297465,0.573737,0.517060,0.170922,0.224717,0.146021,0.082477,0.241085,0.210578,0.062828,0.274294,0.098188,0.156060,0.324795,0.429638,0.299766,0.174941,0.622268,0.330753,0.213898,0.534708,0.321506,0.393939
7,0.284869,0.409537,0.302052,0.159618,0.674099,0.533157,0.435567,0.464861,0.651515,0.504002,0.070541,0.141841,0.077699,0.032720,0.136588,0.246102,0.089722,0.232241,0.190634,0.098607,0.268943,0.498667,0.277852,0.136183,0.654626,0.497531,0.430511,0.707904,0.554504,0.342123
8,0.259312,0.484613,0.277659,0.140997,0.595558,0.675480,0.532568,0.424602,0.489899,0.683867,0.067391,0.273780,0.060406,0.032010,0.184791,0.525115,0.195530,0.271263,0.140823,0.317331,0.254714,0.763859,0.235271,0.129326,0.753682,1.000000,0.882588,0.759450,0.552139,1.000000
9,0.427801,0.457558,0.407090,0.277540,0.265686,0.145114,0.077296,0.165159,0.236364,0.147641,0.097049,0.182726,0.080526,0.062959,0.078730,0.052701,0.027803,0.143796,0.094529,0.074182,0.400569,0.582623,0.365506,0.237122,0.309912,0.124002,0.116534,0.342784,0.272620,0.193362


In [250]:
# View the dataframe in range -1 to 1
df_normalized_m1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,0.286289,-0.454853,0.231567,0.003181,-0.420240,-0.636464,-0.592784,-0.302485,-0.240404,-0.717355,-0.687127,-0.834821,-0.751119,-0.748680,-0.761227,-0.837354,-0.906061,-0.492328,-0.830922,-0.817780,0.213803,-0.392857,0.079635,-0.129571,-0.304893,-0.690873,-0.614058,0.278351,-0.532821,-0.554244
1,0.202991,-0.219479,0.191486,-0.101166,0.028618,-0.137967,-0.074977,0.271372,0.019192,-0.577506,-0.540757,-0.811395,-0.639259,-0.674156,-0.698338,-0.432091,-0.806465,-0.220307,-0.588619,-0.745989,0.112771,-0.279851,0.016883,-0.250983,-0.032820,-0.229250,-0.280511,0.670103,-0.192588,-0.573134
2,-0.579819,-0.278323,-0.532997,-0.794189,0.622642,0.622723,0.131209,0.045726,0.552525,1.000000,-0.721818,-0.648250,-0.746690,-0.923690,-0.497094,0.086430,-0.714091,-0.292669,0.456295,-0.425590,-0.503380,-0.228145,-0.517307,-0.811984,0.830945,0.628023,0.097284,0.769759,1.000000,0.547422
3,0.259785,-0.686845,0.261972,-0.021421,-0.139298,-0.304214,-0.072165,0.036779,-0.243434,-0.626369,-0.532356,-0.813870,-0.558875,-0.672625,-0.335282,-0.664163,-0.712727,-0.285850,-0.727641,-0.708401,0.039488,-0.752132,0.013895,-0.316850,-0.125272,-0.655170,-0.361022,0.116838,-0.684999,-0.714810
4,-0.482323,-0.594860,-0.464032,-0.716988,0.357227,-0.076008,-0.260544,-0.195924,0.037374,0.102359,-0.838494,-0.765736,-0.862413,-0.923840,-0.605874,-0.531379,-0.814545,-0.569237,-0.612540,-0.710681,-0.463536,-0.374733,-0.472185,-0.726504,0.425477,-0.034433,-0.144569,0.196564,-0.045929,-0.090122
5,0.066686,-0.305377,0.047751,-0.239449,-0.241672,-0.450218,-0.471884,-0.264414,-0.258586,-0.685762,-0.757233,-0.817450,-0.771663,-0.824026,-0.823163,-0.826238,-0.886162,-0.606365,-0.836551,-0.911267,0.063678,-0.166311,0.023856,-0.301612,-0.035462,-0.553104,-0.395527,0.327835,-0.409422,-0.624295
6,-0.363056,-0.247886,-0.358579,-0.631474,0.196533,-0.109748,-0.561106,-0.405070,0.147475,0.034120,-0.658157,-0.550566,-0.707958,-0.835046,-0.517830,-0.578845,-0.874343,-0.451411,-0.803625,-0.687879,-0.350409,-0.140725,-0.400468,-0.650118,0.244535,-0.338495,-0.572204,0.069416,-0.356988,-0.212121
7,-0.430262,-0.180927,-0.395895,-0.680764,0.348199,0.066315,-0.128866,-0.070278,0.303030,0.008003,-0.858917,-0.716319,-0.844603,-0.934561,-0.726825,-0.507796,-0.820556,-0.535518,-0.618731,-0.802786,-0.462113,-0.002665,-0.444295,-0.727635,0.309252,-0.004938,-0.138978,0.415808,0.109008,-0.315755
8,-0.481376,-0.030774,-0.444682,-0.718006,0.191117,0.350960,0.065136,-0.150795,-0.020202,0.367734,-0.865218,-0.452440,-0.879188,-0.935980,-0.630418,0.050230,-0.608939,-0.457473,-0.718354,-0.365339,-0.490573,0.527719,-0.529459,-0.741349,0.507363,1.000000,0.765176,0.518900,0.104278,1.000000
9,-0.144399,-0.084883,-0.185820,-0.444920,-0.468629,-0.709772,-0.845408,-0.669682,-0.527273,-0.704718,-0.805903,-0.634547,-0.838948,-0.874082,-0.842540,-0.894598,-0.944394,-0.712408,-0.810942,-0.851637,-0.198862,0.165245,-0.268987,-0.525757,-0.380176,-0.751996,-0.766933,-0.314433,-0.454760,-0.613276


In [251]:
X_train, X_test, y_train, y_test = train_test_split(df_normalized, diagnosis, shuffle=True)
X_train_m1, X_test_m1, y_train_m1, y_test_m1 = train_test_split(df_normalized_m1, diagnosis, shuffle=True)

In [252]:
dataset_train = tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values))
dataset_train_m1 = tf.data.Dataset.from_tensor_slices((X_train_m1.values, y_train_m1.values))

In [253]:
dataset_test = tf.data.Dataset.from_tensor_slices((X_test.values, y_test.values))
dataset_test_m1 = tf.data.Dataset.from_tensor_slices((X_test_m1.values, y_test.values))

In [254]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
        print ('Features: {}, Diagnosis: {}'.format(batch.numpy(), label))

In [255]:
show_batch(dataset_train)

Features: [0.402717 0.346973 0.405017 0.255016 0.572086 0.419668 0.443065 0.452932
 0.438889 0.368155 0.193916 0.214551 0.184611 0.112268 0.18887  0.202992
 0.125556 0.310475 0.149688 0.115294 0.472074 0.463486 0.456646 0.288488
 0.640098 0.353164 0.44353  0.728866 0.319732 0.307359], Diagnosis: 1


In [256]:
show_batch(dataset_train_m1)

Features: [-0.652705  0.049036 -0.665262 -0.827359 -0.206644 -0.675112 -0.888519
 -0.839463 -0.154545 -0.4385   -0.899909 -0.499293 -0.930264 -0.96325
 -0.627698 -0.878361 -0.945404 -0.763402 -0.61451  -0.880865 -0.692636
  0.235075 -0.725385 -0.867037  0.03982  -0.781684 -0.820288 -0.578282
 -0.273014 -0.653286], Diagnosis: 0
